In [17]:
import pandas as pd
import numpy as np
import os,sys
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


os.chdir('/data/arpit.goel/33_GrocerySales/')

store={}
for file in ['train_x','train_y','val_x','val_y','test_x']:
    store[file]=pd.read_hdf('06.Master/02.DataStore.h5', file)
    store[file].columns=map(str,store[file].columns.tolist())

for file in ['train_x','val_x','test_x']:
    store[file]=store[file].fillna(0)

sample=np.random.randint(0, high=len(store['train_x']), size=int(len(store['train_x'])/100))
store['train_x_sample']=store['train_x'].iloc[sample,:]
store['train_y_sample']=store['train_y'].iloc[sample,:]


Opening 06.Master/02.DataStore.h5 in read-only mode
Opening 06.Master/02.DataStore.h5 in read-only mode
Opening 06.Master/02.DataStore.h5 in read-only mode
Opening 06.Master/02.DataStore.h5 in read-only mode
Opening 06.Master/02.DataStore.h5 in read-only mode


In [6]:
import scipy 
scipy.__version__

'0.18.1'

In [2]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.7.0_45"; OpenJDK Runtime Environment (rhel-2.4.3.3.el6-x86_64 u45-b15); OpenJDK 64-Bit Server VM (build 24.45-b08, mixed mode)
  Starting server from /data/arpit.goel/anaconda/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpo4xplmz8
  JVM stdout: /tmp/tmpo4xplmz8/h2o_arpit_goel2_started_from_python.out
  JVM stderr: /tmp/tmpo4xplmz8/h2o_arpit_goel2_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster version:,3.13.0.3923
H2O cluster version age:,6 months and 16 days !!!
H2O cluster name:,H2O_from_python_arpit_goel2_qwacko
H2O cluster total nodes:,1
H2O cluster free memory:,26.52 Gb
H2O cluster total cores:,24
H2O cluster allowed cores:,24
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [3]:
train=h2o.H2OFrame(pd.concat([store['train_x_sample'],store['train_y_sample']],axis=1))
test=h2o.H2OFrame(pd.concat([store['val_x'],store['val_y']],axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
prediction_day="10"

param = {
      "ntrees" : 100
    , "max_depth" : 10
    , "learn_rate" : 0.02
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4241
    , "score_tree_interval": 100
}

from h2o.estimators import H2OXGBoostEstimator
model = H2OXGBoostEstimator(**param)
model.train(x=store['train_x_sample'].columns.tolist(),y=prediction_day,training_frame=train,validation_frame=test)

In [8]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn import model_selection   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from sklearn.metrics import mean_squared_error

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [15]:
def model_train_scikit(reg):
    idv=store['train_x_sample'].columns
    ins_pred=[]
    oos_pred=[]
    ntgt=2
    for i in range(ntgt):
        reg.fit(store['train_x_sample'][idv],store['train_y_sample'].iloc[:,i])
        ins_pred.append(reg.predict(store['train_x'][idv]))
        oos_pred.append(reg.predict(store['val_x'][idv]))
    ins_error=mean_squared_error(store['train_y'].iloc[:,:ntgt].T.values.ravel(),np.array(ins_pred).ravel())
    oos_error=mean_squared_error(store['val_y'].T.values.ravel(),np.array(oos_pred).ravel())
    return ins_error,oos_error

In [18]:
xgb1 = XGBRegressor(learning_rate=0.1, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0,\
 subsample=0.8, colsample_bytree=0.8, objective= 'reg:linear', nthread=20, scale_pos_weight=1, seed=27)
model_train_scikit(xgb1)

ValueError: Found input variables with inconsistent numbers of samples: [21441920, 1340120]

In [21]:
!cat 07.ModelPerformance/04.SGD.csv

model,depth,train_error,validation_error
sgd,3,0.37529161931211136,0.3854975734283082
